In [ ]:
import numpy as np
import chirplet as ch
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import librosa.util
import librosa
import sys
import joblib
import os
import time
import math
import scipy.io as sio
import scipy.signal as sg
from os.path import join, exists
import soundfile as sf


def enframe(signal, nw, inc, winfunc):
    '''将音频信号转化为帧。
    参数含义：
    signal:原始音频型号
    nw:每一帧的长度(这里指采样点的长度，即采样频率乘以时间间隔)
    inc:相邻帧的间隔（同上定义）
    winfunc窗函数winfunc = signal.hamming(nw)
    '''
    signal_length = len(signal)  # 信号总长度
    if signal_length <= nw:  # 若信号长度小于一个帧的长度，则帧数定义为1
        nf = 1
    else:  # 否则，计算帧的总长度
        nf = int(np.ceil((1.0 * signal_length - nw + inc) / inc))
    pad_length = int((nf - 1) * inc + nw)  # 所有帧加起来总的铺平后的长度
    zeros = np.zeros((pad_length - signal_length,))  # 不够的长度使用0填补，类似于FFT中的扩充数组操作
    pad_signal = np.concatenate((signal, zeros))  # 填补后的信号记为pad_signal
    indices = np.tile(np.arange(0, nw), (nf, 1)) + np.tile(np.arange(0, nf * inc, inc),
                                                           (nw, 1)).T  # 相当于对所有帧的时间点进行抽取，得到nf*nw长度的矩阵
    indices = np.array(indices, dtype=np.int32)  # 将indices转化为矩阵
    frames = pad_signal[indices]  # 得到帧信号
    win = np.tile(winfunc, (nf, 1))  # window窗函数，这里默认取1
    return frames * win  # 返回帧信号矩阵




In [1]:
import librosa
import os

def dsilence(file_path, frame_time, overlap_rate):
    # 去除data的静音区，alfa为能量门限值，frame_length分帧长度,, hop_length帧偏移

    print(file_path)
    # Load in librosa's example audio file at its native sampling rate
    data, sr = librosa.load(file_path, sr=None)
    '''plt.plot(data)
    plt.ylabel('data')
    plt.show()'''
    # frame_time=500 #改变分割的长度，单位ms
    frame_length = int(sr * frame_time / 1000)
    hop_length = int((1 - overlap_rate) * frame_length)

    '''plt.plot(tdata)
    plt.ylabel('tdata')
    plt.show()'''
    edata = data / abs(data).max()  # 对语音进行归一化

    # winfunc = sg.hamming(frame_length)
    # frames=enframe(edata, frame_length, hop_length,winfunc)
    frames = librosa.util.frame(edata, frame_length, hop_length)  # 这个更接近matlab结果,不要重叠
    '''librosa.util.frame:
    Parameters:
    y : np.ndarray [shape=(n,)]

    Time series to frame. Must be one-dimensional and contiguous in memory.

    frame_length : int > 0 [scalar]

    Length of the frame in samples

    hop_length : int > 0 [scalar]

    Number of samples to hop between frames

    Returns:
    y_frames : np.ndarray [shape=(frame_length, N_FRAMES)]'''
    # librosa.util.frame和enframe生成的frames的维数正好是反的，要以分割得到的帧数作为row
    row = frames.shape[1]  # 1591#不同分帧函数这里要换
    col = frames.shape[0]  # 50

    print(row)
    print(col)
    """这里判断出音频的长度后补帧"""
    """

    从这里看起


    """

    # # print('帧数',row,col)
    # Energy = np.zeros((1, row))
    #
    # # 短时能量函数
    # for i in range(0, row - 1):
    #     Energy[0, i] = np.sum(abs(frames[:, i] * frames[:, i]), 0)  # 不同分帧函数这里要换
    #
    # MAX = Energy.max()
    # Ave_Energy = Energy.sum() / row
    # Delete = np.zeros((1, row))
    #
    # # Delete(i)=1 表示第i帧为清音帧
    #
    # for i in range(0, row - 1):
    #     if Energy[0, i] < Ave_Energy * alfa:
    #         Delete[0, i] = 1
    #
    # # 暂存每一帧的数据
    # ds_new = np.zeros((frame_length))
    # # 保存每一帧的数据，横为帧数，纵为帧长
    # ds_data = np.zeros((frame_length, int(row - Delete.sum())))
    #
    # begin = 0
    # for i in range(0, row - 1):
    #     if Delete[0, i] == 0:
    #         for j in range(0, frame_length - 1, 1):
    #             # New[begin*hop_length+j]=edata[i*hop_length+j]
    #             ds_new[j] = edata[i * hop_length + j]
    #         ds_data[:, begin] = ds_new
    #         begin = begin + 1
    # print('Numberofslices:', begin)
    # return ds_data, edata

file_path = r"C:\Users\Lollipop\Desktop\语音数据测试\Track1-positive\AHGDPacO_cough.wav"
dsilence(file_path, 1000, 0.5)

C:\Users\Lollipop\Desktop\语音数据测试\Track1-positive\AHGDPacO_cough.wav
17
44100


C:\Users\Lollipop\AppData\Local\Temp/ipykernel_15376/3162848962.py:24: FutureWarning: Pass frame_length=44100, hop_length=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.util.frame(edata, frame_length, hop_length)  # 这个更接近matlab结果,不要重叠
